In [38]:
import pandas as pd
device_df = pd.read_csv('device.csv')
psycho_df = pd.read_csv('psychometric.csv', index_col=0)

In [39]:
import math
import random
device_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
device_df["insider"] = device_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(device_df[device_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
device_df = device_df.drop(del_rows)
device_df

,id,date,user,pc,activity,insider
3,{H0Z7-E6GB57XZ-1603MOXD},01/02/2010 07:59:49,IIW0249,PC-0843,Connect,1
4,{L7P2-G4PX02RX-7999GYOY},01/02/2010 08:04:26,IIW0249,PC-0843,Disconnect,1
5,{B4V1-F2XA86NJ-0683CLUB},01/02/2010 08:17:35,HPH0075,PC-2417,Disconnect,1
7,{J3A7-Z5JP56DF-7855PJKX},01/02/2010 08:25:18,RRC0553,PC-6672,Connect,1
8,{H9X8-R0QB50BG-6009OAKA},01/02/2010 08:25:19,MOH0273,PC-6699,Connect,1
...,...,...,...,...,...,...
405360,{Y4G7-A9RT08ZD-3256TOXA},05/16/2011 19:45:31,HRL0540,PC-1117,Connect,1
405362,{C8L6-I7DO58KD-8877WGSR},05/16/2011 20:02:35,HRL0540,PC-1117,Disconnect,1
405369,{S2F0-D5DT50VX-9735HVPB},05/16/2011 20:54:37,BRM0995,PC-0768,Connect,1
405370,{Q2Q0-I9YT91CF-7485JWFK},05/16/2011 20:56:09,KCO0122,PC-9174,Disconnect,1


In [40]:
device_df["activity"] = device_df["activity"].apply(lambda x : 0 if x == "Connect" else  1) # activityを0, 1に変換
device_df["date"] = pd.to_datetime(device_df["date"]) # 日時をdatetime型に変換

In [46]:
def get_timezone(hour: int) -> int:
        if 0 <= hour and hour < 6:
            return 0
        elif 6 <= hour and hour < 12:
            return 1
        elif 12 <= hour and hour < 18:
            return 2
        else:
            return 3

users = device_df['user'].unique()
d = {}
for user in users:
    d[user] = {} # connect, disconnectを記録
    for index, row in device_df[device_df['user']== user].iterrows():
        ym = str(row['date'].year)+str(row['date'].month)
        hour = row['date'].hour
        if(row['activity'] == 0):
            if ym not in d[user]:
                d[user][ym] = [0]*8
            tz = get_timezone(hour)
            d[user][ym][tz] += 1 
        if(row['activity'] == 1):
            if ym not in d[user]:
                d[user][ym] = [0]*8
            tz = get_timezone(hour)
            d[user][ym][tz+4] += 1 

In [72]:
# USBの挿抜に関する列を追加

main_df = pd.DataFrame({'user_id':[], 'O': [], 'C': [], 'E': [], 'A': [], 'N': [], 
                        'dev_con_0to6': [], 'dev_con_6to12': [], 'dev_con_12to18': [], 'dev_con_18to24': [],
                        'dev_dis_0to6': [], 'dev_dis_6to12': [], 'dev_dis_12to18': [], 'dev_dis_18to24': [], 
                       }, index=[])

for user, v in d.items():
    for ym, vv in v.items():
        row = psycho_df[psycho_df['user_id']==user]
        main_df = main_df.append({'user_id': user, 'O': row['O'][0], 'C':  row['C'][0], 'E':  row['E'][0], 'A':  row['A'][0], 'N':  row['N'][0],  
                        'dev_con_0to6':  vv[0], 'dev_con_6to12': vv[1], 'dev_con_12to18': vv[2], 'dev_con_18to24': vv[3],
                        'dev_dis_0to6': vv[4], 'dev_dis_6to12': vv[5], 'dev_dis_12to18': vv[6], 'dev_dis_18to24': vv[7], 
                       }, ignore_index=True)
main_df

,user_id,O,C,E,A,N,dev_con_0to6,dev_con_6to12,dev_con_12to18,dev_con_18to24,dev_dis_0to6,dev_dis_6to12,dev_dis_12to18,dev_dis_18to24
0,IIW0249,39.0,36.0,43.0,38.0,28.0,0.0,11.0,12.0,0.0,0.0,9.0,15.0,0.0
1,IIW0249,39.0,36.0,43.0,38.0,28.0,0.0,5.0,8.0,0.0,0.0,6.0,11.0,0.0
2,IIW0249,39.0,36.0,43.0,38.0,28.0,0.0,16.0,15.0,0.0,0.0,13.0,15.0,0.0
3,IIW0249,39.0,36.0,43.0,38.0,28.0,0.0,15.0,15.0,0.0,0.0,7.0,12.0,1.0
4,IIW0249,39.0,36.0,43.0,38.0,28.0,0.0,16.0,12.0,0.0,0.0,13.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3407,KLH0596,30.0,10.0,35.0,12.0,29.0,0.0,1.0,0.0,18.0,0.0,1.0,0.0,18.0
3408,MAR0955,36.0,44.0,23.0,44.0,25.0,7.0,1.0,0.0,13.0,6.0,2.0,0.0,13.0
3409,WDD0366,43.0,42.0,17.0,19.0,28.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0
3410,WDD0366,43.0,42.0,17.0,19.0,28.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [153]:
df_ins = (df["insider"] == 0)
print("insider :", df_ins.sum()) # 内部不正者のログ件数をカウント
df_niins = (df["insider"] == 1)
print("not insider :", df_niins.sum()) # 内部不正者でない人のログ件数をカウント
print(df_ins.sum()/(df_ins.sum()+df_niins.sum())) # 全体における内部不正者のログ件数の割合

insider : 44134
not insider : 46962
0.484477913410029


In [154]:
import datetime
# df['date'] = df['date'].map(datetime.datetime.timestamp) # datetime to float
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['second'] = df['date'].dt.second

In [54]:
# import numpy as np
# # 正規化
# date_mean = np.mean(df['date'])
# date_std =  np.std(df['date'])
# df['date'] = df['date'].apply(lambda x: (x - date_mean) / date_std)
# print(df['date'])

0        -1.663197
1        -1.663116
2        -1.663012
3        -1.663009
4        -1.662987
            ...   
405375    1.824592
405376    1.824671
405377    1.824695
405378    1.824859
405379    1.824859
Name: date, Length: 405380, dtype: float64


In [155]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = df[['year', 'month', 'day', 'hour', 'minute', 'second', 'activity']]
Y = df['insider']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする

# ロジスティック回帰で学習
lr = LogisticRegression()
lr.fit(X_train, Y_train)
print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)

Y_pred = lr.predict(X_test)
np.set_printoptions(threshold=np.inf)
print(Y_pred)

coefficient =  [[-4.41249688e-05 -3.81730839e-03 -4.88289177e-04  1.23375726e-02
   3.46066486e-05  8.99787266e-04 -5.26120032e-04]]
intercept =  [-1.03865534e-05]
[1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1
 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 

In [156]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# モデルの評価
print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

confusion matrix = 
 [[ 998 7836]
 [ 692 8694]]
accuracy =  0.5319429198682766
precision =  0.5259528130671506
recall =  0.926273172810569
f1 score =  0.6709368729742244
